In [5]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

In [7]:
%autoreload
from pathlib import Path
import sys
root = Path().resolve().absolute().parent.parent
print(root)
sys.path.append(str(root))

from src.pipeline import Pipeline, BuilingIdsEnum
pipe = Pipeline()



/Users/matsalexander/Desktop/SolarEnergyImpact


In [8]:
building_a = pipe.get_data(BuilingIdsEnum.A)
building_b = pipe.get_data(BuilingIdsEnum.B)
building_c = pipe.get_data(BuilingIdsEnum.C)
combined_df = pd.concat([building_a, building_b, building_c])
# Perform the train-test split with stratification based on 'building_id'
train_data, test_data = train_test_split(
    combined_df,
    test_size=0.20,
    random_state=42,
    stratify=combined_df['building']
)

# select features
features = ["timestamp", "area", "temperature", "wind_speed", "cloud_fraction", "precipitation"]
target = "value_import"

train_data = train_data[features + [target]]
test_data = test_data[features + [target]]


In [ ]:
# Train the model using AutoGluon
predictor = TabularPredictor(label=target, eval_metric='mean_absolute_error').fit(
    train_data, 
    # presets='best_quality',
    presets="medium_quality_faster_train",
    excluded_model_types=['KNN']
    )

No path specified. Models will be saved in: "AutogluonModels/ag-20241109_030847"
Preset alias specified: 'medium_quality_faster_train' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.13
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.6.0: Mon Jul 29 21:14:30 PDT 2024; root:xnu-10063.141.2~1/RELEASE_ARM64_T6000
CPU Count:          10
Memory Avail:       14.70 GB / 32.00 GB (45.9%)
Disk Space Avail:   622.90 GB / 926.35 GB (67.2%)
Presets specified: ['medium_quality_faster_train']
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20241109_030847"
Train Data Rows:    24880
Train Data Columns: 6
Label Column:       value_import
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stdde

[1000]	valid_set's l1: 6.06772
[2000]	valid_set's l1: 5.7793
[3000]	valid_set's l1: 5.59873
[4000]	valid_set's l1: 5.48377
[5000]	valid_set's l1: 5.40339
[6000]	valid_set's l1: 5.33925
[7000]	valid_set's l1: 5.29278
[8000]	valid_set's l1: 5.24658
[9000]	valid_set's l1: 5.20613
[10000]	valid_set's l1: 5.18081


	-5.1808	 = Validation score   (-mean_absolute_error)
	43.01s	 = Training   runtime
	0.33s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's l1: 4.5667
[2000]	valid_set's l1: 4.4089
[3000]	valid_set's l1: 4.30856
[4000]	valid_set's l1: 4.25958
[5000]	valid_set's l1: 4.23767
[6000]	valid_set's l1: 4.22371
[7000]	valid_set's l1: 4.21315
[8000]	valid_set's l1: 4.20854
[9000]	valid_set's l1: 4.20906
[10000]	valid_set's l1: 4.20779


	-4.2056	 = Validation score   (-mean_absolute_error)
	94.19s	 = Training   runtime
	0.34s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-3.5728	 = Validation score   (-mean_absolute_error)
	2.88s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: CatBoost ...


In [ ]:
# Evaluate on test data
performance = predictor.evaluate(test_data)
# best model: ag-20241022_161331

print("Evaluation Performance:")
print(performance)  # This will show various metrics such as R^2, RMSE, etc.

# To see feature importance
global_importance = predictor.feature_importance(test_data)
print("\nFeature Importance:")
print(global_importance)  # Shows which features had the most impact on model predictions

Computing feature importance via permutation shuffling for 3 features using 5000 rows with 5 shuffle sets...


Evaluation Performance:
{'mean_absolute_error': -3.1217798479004197, 'root_mean_squared_error': -4.916806183789267, 'mean_squared_error': -24.174983048948373, 'r2': 0.8285978268071899, 'pearsonr': 0.9103643086413618, 'median_absolute_error': -1.762607803344725}


	194.55s	= Expected runtime (38.91s per shuffle set)
	162.11s	= Actual runtime (Completed 5 of 5 shuffle sets)



Feature Importance:
             importance    stddev       p_value  n  p99_high   p99_low
timestamp      5.904354  0.120787  2.100541e-08  5  6.153056  5.655651
area           5.713944  0.122575  2.539637e-08  5  5.966327  5.461561
temperature    5.251591  0.101069  1.645395e-08  5  5.459692  5.043489


In [ ]:
# model location => AutogluonModels/ag-20241016_095906
main_building = pipe.get_data(BuilingIdsEnum.MAIN)

data_predict = main_building[features]
data_predict


,timestamp,temperature,area
4,2023-07-01 00:00:00,13.6,1199
8,2023-07-01 01:00:00,13.2,1199
16,2023-07-01 02:00:00,12.3,1199
20,2023-07-01 03:00:00,11.9,1199
29,2023-07-01 04:00:00,11.9,1199
...,...,...,...
54451,2024-09-04 19:00:00,17.5,1199
54456,2024-09-04 20:00:00,17.5,1199
54462,2024-09-04 21:00:00,17.5,1199
54468,2024-09-04 22:00:00,17.5,1199


In [ ]:
prediciton1 = predictor.predict(data_predict)

In [ ]:
# save predicitons as a csv in data folder from root.
from pathlib import Path


prediciton1_df = pd.DataFrame(prediciton1)
my_path = Path().resolve().parent / 'data' / 'prediction_drita.csv'
prediciton1_df.to_csv(my_path, index=False)